In [77]:
import hashlib
import time
import random
import math
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
from IPython.display import display
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [78]:
class StandardBloomFilter:
    def __init__(self, n, fp_rate):
        self.size = self._get_size(n, fp_rate)
        self.hash_count = self._get_hash_count(self.size, n)
        self.bit_array = [0] * self.size

    def _hashes(self, item):
        return [hashlib.sha256(f"{item}{i}".encode()).hexdigest() for i in range(self.hash_count)]

    def _get_size(self, n, p):
        m = -(n * math.log(p)) / (math.log(2)**2)
        return int(m)

    def _get_hash_count(self, m, n):
        return int((m / n) * math.log(2))

    def add(self, item):
        for i in range(self.hash_count):
            idx = int(hashlib.md5(f"{item}{i}".encode()).hexdigest(), 16) % self.size
            self.bit_array[idx] = 1

    def query(self, item):
        for i in range(self.hash_count):
            idx = int(hashlib.md5(f"{item}{i}".encode()).hexdigest(), 16) % self.size
            if self.bit_array[idx] == 0:
                return False
        return True


In [79]:
class CountingBloomFilter:
    def __init__(self, n, fp_rate):
        self.size = self._get_size(n, fp_rate)
        self.hash_count = self._get_hash_count(self.size, n)
        self.count_array = [0] * self.size

    def _get_size(self, n, p):
        return int(-(n * math.log(p)) / (math.log(2)**2))

    def _get_hash_count(self, m, n):
        return int((m / n) * math.log(2))

    def add(self, item):
        for i in range(self.hash_count):
            idx = int(hashlib.md5(f"{item}{i}".encode()).hexdigest(), 16) % self.size
            self.count_array[idx] += 1

    def remove(self, item):
        for i in range(self.hash_count):
            idx = int(hashlib.md5(f"{item}{i}".encode()).hexdigest(), 16) % self.size
            self.count_array[idx] = max(0, self.count_array[idx] - 1)

    def query(self, item):
        for i in range(self.hash_count):
            idx = int(hashlib.md5(f"{item}{i}".encode()).hexdigest(), 16) % self.size
            if self.count_array[idx] == 0:
                return False
        return True


In [80]:

class NeuralNetworkBloomFilter:
    def __init__(self):
        self.model = MLPClassifier(hidden_layer_sizes=(10,), max_iter=200)
        self.set_members = set()

    def _featurize(self, url):
        return np.array([ord(c) for c in url[:50]] + [0] * (50 - len(url))).reshape(1, -1)

    def train(self, positives, negatives):
        X = [self._featurize(x).flatten() for x in positives + negatives]
        y = [1]*len(positives) + [0]*len(negatives)
        self.model.fit(X, y)
        self.set_members = set(positives)

    def add(self, item): pass  # Not used

    def query(self, item):
        x = self._featurize(item)
        pred = self.model.predict(x)[0]
        if pred == 1:
            return True
        return False


In [81]:
class RandomForestBloomFilter:
    def __init__(self, **rf_kwargs):
        # you can pass n_estimators, max_depth, etc.
        self.model = RandomForestClassifier(**rf_kwargs)
        self.set_members = set()

    def _featurize(self, x):
        arr = [ord(c) for c in x[:50]] + [0]*(50 - len(x))
        return np.array(arr).reshape(1, -1)

    def train(self, positives, negatives):
        X = np.vstack([self._featurize(u) for u in positives + negatives])
        y = np.array([1]*len(positives) + [0]*len(negatives))
        self.model.fit(X, y)
        self.set_members = set(positives)

    def add(self, item):
        pass  # not used

    def query(self, item):
        if item in self.set_members:
            return True
        x = self._featurize(item)
        return bool(self.model.predict(x)[0])

In [82]:
class LogisticRegressionBloomFilter:
    def __init__(self, **lr_kwargs):
        self.model = LogisticRegression(**lr_kwargs)
        self.set_members = set()

    def _featurize(self, x):
        arr = [ord(c) for c in x[:50]] + [0]*(50 - len(x))
        return np.array(arr).reshape(1, -1)

    def train(self, positives, negatives):
        X = np.vstack([self._featurize(u) for u in positives + negatives])
        y = np.array([1]*len(positives) + [0]*len(negatives))
        self.model.fit(X, y)
        self.set_members = set(positives)

    def add(self, item):
        pass

    def query(self, item):
        if item in self.set_members:
            return True
        x = self._featurize(item)
        return bool(self.model.predict(x)[0])

In [83]:
class SandwichBloomFilter:
    def __init__(self, ml_filter, positives, negatives, fp_rate_small=0.20):
        """
        ml_filter: any object with .train(positives, negatives) and .query(item)
        """
        self.ml = ml_filter
        self.ml.train(positives, negatives)

        # small classical BF to catch anything the ML lets through
        self.small = StandardBloomFilter(len(positives), fp_rate_small)
        for u in positives:
            self.small.add(u)

    def add(self, item):
        pass

    def query(self, item):
        if not self.ml.query(item):
            return False
        return self.small.query(item)

In [84]:
def get_memory_usage(bf):
    """Estimate memory used by each filter."""
    size = sys.getsizeof(bf)

    # classical bit/count arrays
    if hasattr(bf, 'bit_array'):
        size += sys.getsizeof(bf.bit_array)
    if hasattr(bf, 'count_array'):
        size += sys.getsizeof(bf.count_array)

    # Sandwich: just sum its two parts
    if hasattr(bf, 'small') and hasattr(bf, 'ml'):
        return get_memory_usage(bf.ml) + get_memory_usage(bf.small)

    # any ML‐based filter with a .model
    if hasattr(bf, 'model'):
        mdl = bf.model

        # neural networks & logistic regression have coefs_ / intercepts_
        if hasattr(mdl, 'coefs_'):
            for coef in mdl.coefs_:
                size += coef.nbytes
            for intercept in mdl.intercepts_:
                size += intercept.nbytes

        # random forests have many tree estimators
        if hasattr(mdl, 'estimators_'):
            for tree_est in mdl.estimators_:
                tree = tree_est.tree_
                # pick the big arrays inside each tree
                for arr_name in ('threshold', 'feature', 'children_left',
                                 'children_right', 'value'):
                    arr = getattr(tree, arr_name, None)
                    if isinstance(arr, np.ndarray):
                        size += arr.nbytes

    return size


my_set = set()

for i in range(40000):
    my_set.add(i)
    
print(get_memory_usage(my_set))

In [85]:
def evaluate(bf, positives, negatives):
    """Run insertions and then measure FP, FN (for NN), timing, throughput, memory."""
    # — insert positives —
    for url in positives:
        bf.add(url)

    # — measure false positives —
    start = time.time()
    false_positives = sum(1 for url in negatives if bf.query(url) and url not in positives)
    elapsed = time.time() - start

    # — measure false negatives (only makes sense for NN-based filters) —
    false_negatives = 0
    if isinstance(bf, NeuralNetworkBloomFilter) or isinstance(bf, RandomForestBloomFilter) or isinstance(bf, LogisticRegressionBloomFilter) :
        for url in positives:
            if not bf.query(url):
                false_negatives += 1
    fnr = false_negatives / len(positives) if positives else 0.0

    # — compute metrics —
    fpr = false_positives / len(negatives)
    avg_query_time = elapsed / len(negatives)
    throughput = len(negatives) / elapsed if elapsed > 0 else float('inf')
    mem_bytes = get_memory_usage(bf)

    # — print a summary —
    print(f"\n=== {bf.__class__.__name__} ===")
    print(f"Memory Usage:             {mem_bytes:,} bytes")
    print(f"False‐Positive Rate:      {fpr:.4%}")
    if isinstance(bf, NeuralNetworkBloomFilter) or isinstance(bf, RandomForestBloomFilter) or isinstance(bf, LogisticRegressionBloomFilter) :
        print(f"False‐Negative Rate:      {fnr:.4%}")
    print(f"Avg Query Time:           {avg_query_time:.6f} s")
    print(f"Throughput:               {throughput:,.0f} queries/s")

    return {
        'fpr': fpr,
        'fnr': fnr if isinstance(bf, NeuralNetworkBloomFilter) or isinstance(bf, RandomForestBloomFilter) or isinstance(bf, LogisticRegressionBloomFilter) else None,
        'avg_time': avg_query_time,
        'throughput': throughput,
        'memory_bytes': mem_bytes
    }

In [86]:
# 🌐 URL Dataset
print("\U0001f310 URL Dataset Bloom Filter Evaluation\n")
with open("../datasets/urls/url_positives.txt") as f:
    positives = f.read().splitlines()
with open("../datasets/urls/url_negatives.txt") as f:
    negatives = f.read().splitlines()

ml_filters = [
    ("NeuralNet",      NeuralNetworkBloomFilter()),
    ("RandomForest",   RandomForestBloomFilter(n_estimators=100)),
    ("LogisticRegr",   LogisticRegressionBloomFilter(max_iter=200)),
]

filters = [
    ("Standard", StandardBloomFilter(len(positives), 0.10)),
    ("Counting", CountingBloomFilter(len(positives), 0.10)),
]
# add the plain ML filters
filters += [(name, bf) for name, bf in ml_filters]
# add a sandwich for each ML filter
filters += [
    (f"Sandwich-{name}", SandwichBloomFilter(bf, positives, negatives, 0.20))
    for name, bf in ml_filters
]

results_url = []
for name, bf in filters:
    m = evaluate(bf, positives, negatives)
    results_url.append({
        "Filter":            name,
        "Mem (bytes)":       m["memory_bytes"],
        "FP Rate":           m["fpr"],
        "FN Rate":           m["fnr"],
        "Avg Time (s)":      m["avg_time"],
        "Throughput (q/s)":  m["throughput"],
    })

df_url = pd.DataFrame(results_url)
display(df_url)

🌐 URL Dataset Bloom Filter Evaluation



/Users/milesbramwit/cs6386/finalProject/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



=== StandardBloomFilter ===
Memory Usage:             1,708,776 bytes
False‐Positive Rate:      10.1150%
Avg Query Time:           0.000044 s
Throughput:               22,638 queries/s

=== CountingBloomFilter ===
Memory Usage:             1,708,776 bytes
False‐Positive Rate:      10.1150%
Avg Query Time:           0.000044 s
Throughput:               22,645 queries/s

=== NeuralNetworkBloomFilter ===
Memory Usage:             4,224 bytes
False‐Positive Rate:      31.2600%
False‐Negative Rate:      20.5897%
Avg Query Time:           0.000217 s
Throughput:               4,612 queries/s

=== RandomForestBloomFilter ===
Memory Usage:             133,940,792 bytes
False‐Positive Rate:      0.0000%
False‐Negative Rate:      0.0000%
Avg Query Time:           0.004179 s
Throughput:               239 queries/s

=== LogisticRegressionBloomFilter ===
Memory Usage:             56 bytes
False‐Positive Rate:      30.6225%
False‐Negative Rate:      0.0000%
Avg Query Time:           0.000232 s
Throu

,Filter,Mem (bytes),FP Rate,FN Rate,Avg Time (s),Throughput (q/s)
0,Standard,1708776,0.101150,NaN,0.000044,22637.672504
1,Counting,1708776,0.101150,NaN,0.000044,22645.021021
2,NeuralNet,4224,0.312600,0.205897,0.000217,4611.776539
3,RandomForest,133940792,0.000000,0.000000,0.004179,239.268726
4,LogisticRegr,56,0.306225,0.000000,0.000232,4311.381837
5,Sandwich-NeuralNet,1198640,0.063000,NaN,0.000113,8825.994732
6,Sandwich-RandomForest,135135208,0.000000,NaN,0.004048,247.011764
7,Sandwich-LogisticRegr,1194472,0.061825,NaN,0.000104,9574.823282


In [87]:
# 🔐 Password Dataset
print("\U0001f510 Password Dataset Bloom Filter Evaluation\n")
with open("../datasets/passwords/password_positives.txt") as f:
    positives = f.read().splitlines()
with open("../datasets/passwords/password_negatives.txt") as f:
    negatives = f.read().splitlines()

filters_pw = [
    ("Standard", StandardBloomFilter(len(positives), 0.0005)),
    ("Counting", CountingBloomFilter(len(positives), 0.0005)),
] + [(name, bf) for name, bf in ml_filters] + [
    (f"Sandwich-{name}", SandwichBloomFilter(bf, positives, negatives, 0.20))
    for name, bf in ml_filters
]

results_pw = []
for name, bf in filters_pw:
    m = evaluate(bf, positives, negatives)
    results_pw.append({
        "Filter":            name,
        "Mem (bytes)":       m["memory_bytes"],
        "FP Rate":           m["fpr"],
        "FN Rate":           m["fnr"],
        "Avg Time (s)":      m["avg_time"],
        "Throughput (q/s)":  m["throughput"],
    })

df_pw = pd.DataFrame(results_pw)
display(df_pw)


🔐 Password Dataset Bloom Filter Evaluation


=== StandardBloomFilter ===
Memory Usage:             1,265,728 bytes
False‐Positive Rate:      0.0200%
Avg Query Time:           0.000002 s
Throughput:               427,210 queries/s

=== CountingBloomFilter ===
Memory Usage:             1,265,728 bytes
False‐Positive Rate:      0.0200%
Avg Query Time:           0.000003 s
Throughput:               388,498 queries/s

=== NeuralNetworkBloomFilter ===
Memory Usage:             4,224 bytes
False‐Positive Rate:      0.2200%
False‐Negative Rate:      0.0600%
Avg Query Time:           0.000083 s
Throughput:               12,096 queries/s

=== RandomForestBloomFilter ===
Memory Usage:             245,912 bytes
False‐Positive Rate:      0.0000%
False‐Negative Rate:      0.0000%
Avg Query Time:           0.003468 s
Throughput:               288 queries/s

=== LogisticRegressionBloomFilter ===
Memory Usage:             56 bytes
False‐Positive Rate:      1.9200%
False‐Negative Rate:      0.0000%
Avg 

,Filter,Mem (bytes),FP Rate,FN Rate,Avg Time (s),Throughput (q/s)
0,Standard,1265728,0.0002,NaN,0.000002,427209.892136
1,Counting,1265728,0.0002,NaN,0.000003,388498.175284
2,NeuralNet,4224,0.0022,0.0006,0.000083,12096.199635
3,RandomForest,245912,0.0000,0.0000,0.003468,288.330806
4,LogisticRegr,56,0.0192,0.0000,0.000073,13691.712022
5,Sandwich-NeuralNet,272320,0.0003,NaN,0.000092,10833.487877
6,Sandwich-RandomForest,514008,0.0000,NaN,0.003475,287.775869
7,Sandwich-LogisticRegr,268152,0.0036,NaN,0.000068,14806.479501


In [88]:
# 📏 IP Address Dataset
print("\U0001f4cf IP Address Dataset Bloom Filter Evaluation\n")
with open("../datasets/ip_addresses/ip_address_positives.txt") as f:
    positives = f.read().splitlines()
with open("../datasets/ip_addresses/ip_addresses_negatives.txt") as f:
    negatives = f.read().splitlines()

filters_ip = [
    ("Standard", StandardBloomFilter(len(positives), 0.01)),
    ("Counting", CountingBloomFilter(len(positives), 0.01)),
] + [(name, bf) for name, bf in ml_filters] + [
    (f"Sandwich-{name}", SandwichBloomFilter(bf, positives, negatives, 0.20))
    for name, bf in ml_filters
]

results_ip = []
for name, bf in filters_ip:
    m = evaluate(bf, positives, negatives)
    results_ip.append({
        "Filter":            name,
        "Mem (bytes)":       m["memory_bytes"],
        "FP Rate":           m["fpr"],
        "FN Rate":           m["fnr"],
        "Avg Time (s)":      m["avg_time"],
        "Throughput (q/s)":  m["throughput"],
    })

df_ip = pd.DataFrame(results_ip)
display(df_ip)


📏 IP Address Dataset Bloom Filter Evaluation



/Users/milesbramwit/cs6386/finalProject/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/milesbramwit/cs6386/finalProject/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



=== StandardBloomFilter ===
Memory Usage:             3,067,328 bytes
False‐Positive Rate:      0.9625%
Avg Query Time:           0.000007 s
Throughput:               142,570 queries/s

=== CountingBloomFilter ===
Memory Usage:             3,067,328 bytes
False‐Positive Rate:      0.9625%
Avg Query Time:           0.000007 s
Throughput:               143,614 queries/s

=== NeuralNetworkBloomFilter ===
Memory Usage:             4,224 bytes
False‐Positive Rate:      12.0700%
False‐Negative Rate:      12.8925%
Avg Query Time:           0.000148 s
Throughput:               6,738 queries/s

=== RandomForestBloomFilter ===
Memory Usage:             44,123,000 bytes
False‐Positive Rate:      0.0000%
False‐Negative Rate:      0.0000%
Avg Query Time:           0.003678 s
Throughput:               272 queries/s

=== LogisticRegressionBloomFilter ===
Memory Usage:             56 bytes
False‐Positive Rate:      16.0550%
False‐Negative Rate:      0.0000%
Avg Query Time:           0.000148 s
Throug

,Filter,Mem (bytes),FP Rate,FN Rate,Avg Time (s),Throughput (q/s)
0,Standard,3067328,0.009625,NaN,0.000007,142569.930768
1,Counting,3067328,0.009625,NaN,0.000007,143613.743325
2,NeuralNet,4224,0.120700,0.128925,0.000148,6738.310515
3,RandomForest,44123000,0.000000,0.000000,0.003678,271.915739
4,LogisticRegr,56,0.160550,0.000000,0.000148,6757.327349
5,Sandwich-NeuralNet,1076280,0.024675,NaN,0.000094,10652.463734
6,Sandwich-RandomForest,45195056,0.000000,NaN,0.003469,288.229624
7,Sandwich-LogisticRegr,1072112,0.032600,NaN,0.000084,11939.564676


In [89]:
# 📞 Phone Number Dataset
print("\U0001f4de Phone Number Dataset Bloom Filter Evaluation\n")
with open("../datasets/phone_numbers/phone_numbers_positives.txt") as f:
    positives = f.read().splitlines()
with open("../datasets/phone_numbers/phone_numbers_negatives.txt") as f:
    negatives = f.read().splitlines()

filters_phone = [
    ("Standard", StandardBloomFilter(len(positives), 0.01)),
    ("Counting", CountingBloomFilter(len(positives), 0.01)),
] + [(name, bf) for name, bf in ml_filters] + [
    (f"Sandwich-{name}", SandwichBloomFilter(bf, positives, negatives, 0.20))
    for name, bf in ml_filters
]

results_phone = []
for name, bf in filters_phone:
    m = evaluate(bf, positives, negatives)
    results_phone.append({
        "Filter":            name,
        "Mem (bytes)":       m["memory_bytes"],
        "FP Rate":           m["fpr"],
        "FN Rate":           m["fnr"],
        "Avg Time (s)":      m["avg_time"],
        "Throughput (q/s)":  m["throughput"],
    })

df_phone = pd.DataFrame(results_phone)
display(df_phone)

📞 Phone Number Dataset Bloom Filter Evaluation



/Users/milesbramwit/cs6386/finalProject/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(



=== StandardBloomFilter ===
Memory Usage:             62,528 bytes
False‐Positive Rate:      1.5000%
Avg Query Time:           0.000002 s
Throughput:               439,516 queries/s

=== CountingBloomFilter ===
Memory Usage:             62,528 bytes
False‐Positive Rate:      1.5000%
Avg Query Time:           0.000002 s
Throughput:               433,654 queries/s

=== NeuralNetworkBloomFilter ===
Memory Usage:             4,224 bytes
False‐Positive Rate:      37.4000%
False‐Negative Rate:      31.9410%
Avg Query Time:           0.000083 s
Throughput:               12,099 queries/s

=== RandomForestBloomFilter ===
Memory Usage:             4,042,712 bytes
False‐Positive Rate:      0.0000%
False‐Negative Rate:      0.0000%
Avg Query Time:           0.003559 s
Throughput:               281 queries/s

=== LogisticRegressionBloomFilter ===
Memory Usage:             56 bytes
False‐Positive Rate:      26.4000%
False‐Negative Rate:      0.0000%
Avg Query Time:           0.000068 s
Throughput: 

,Filter,Mem (bytes),FP Rate,FN Rate,Avg Time (s),Throughput (q/s)
0,Standard,62528,0.015,NaN,0.000002,439516.294666
1,Counting,62528,0.015,NaN,0.000002,433654.259719
2,NeuralNet,4224,0.374,0.31941,0.000083,12098.593215
3,RandomForest,4042712,0.000,0.00000,0.003559,280.987910
4,LogisticRegr,56,0.264,0.00000,0.000068,14666.167806
5,Sandwich-NeuralNet,26144,0.072,NaN,0.000081,12285.132669
6,Sandwich-RandomForest,4064632,0.000,NaN,0.003690,271.011899
7,Sandwich-LogisticRegr,21976,0.057,NaN,0.000068,14744.016170


In [93]:
# 📧 Email Dataset
print("\U0001f4e7 Email Dataset Bloom Filter Evaluation\n")
with open("../datasets/emails/spam_email_positives.txt") as f:
    positives = f.read().splitlines()
with open("../datasets/emails/spam_email_negatives.txt") as f:
    negatives = f.read().splitlines()

filters_email = [
    ("Standard", StandardBloomFilter(len(positives), 0.01)),
    ("Counting", CountingBloomFilter(len(positives), 0.01)),
] + [(name, bf) for name, bf in ml_filters] + [
    (f"Sandwich-{name}", SandwichBloomFilter(bf, positives, negatives, 0.20))
    for name, bf in ml_filters
]

results_email = []
for name, bf in filters_email:
    m = evaluate(bf, positives, negatives)
    results_email.append({
        "Filter":            name,
        "Mem (bytes)":       m["memory_bytes"],
        "FP Rate":           m["fpr"],
        "FN Rate":           m["fnr"],
        "Avg Time (s)":      m["avg_time"],
        "Throughput (q/s)":  m["throughput"],
    })

df_email = pd.DataFrame(results_email)
display(df_email)

📧 Email Dataset Bloom Filter Evaluation



/Users/milesbramwit/cs6386/finalProject/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/milesbramwit/cs6386/finalProject/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



=== StandardBloomFilter ===
Memory Usage:             52,712 bytes
False‐Positive Rate:      1.6012%
Avg Query Time:           0.000002 s
Throughput:               445,017 queries/s

=== CountingBloomFilter ===
Memory Usage:             52,712 bytes
False‐Positive Rate:      1.6012%
Avg Query Time:           0.000003 s
Throughput:               383,840 queries/s

=== NeuralNetworkBloomFilter ===
Memory Usage:             4,224 bytes
False‐Positive Rate:      5.9680%
False‐Negative Rate:      11.5160%
Avg Query Time:           0.000083 s
Throughput:               11,990 queries/s

=== RandomForestBloomFilter ===
Memory Usage:             892,952 bytes
False‐Positive Rate:      0.0000%
False‐Negative Rate:      0.0000%
Avg Query Time:           0.003516 s
Throughput:               284 queries/s

=== LogisticRegressionBloomFilter ===
Memory Usage:             56 bytes
False‐Positive Rate:      5.8224%
False‐Negative Rate:      0.0000%
Avg Query Time:           0.000067 s
Throughput:     

,Filter,Mem (bytes),FP Rate,FN Rate,Avg Time (s),Throughput (q/s)
0,Standard,52712,0.016012,NaN,0.000002,445017.273822
1,Counting,52712,0.016012,NaN,0.000003,383839.995737
2,NeuralNet,4224,0.059680,0.11516,0.000083,11990.008688
3,RandomForest,892952,0.000000,0.00000,0.003516,284.374916
4,LogisticRegr,56,0.058224,0.00000,0.000067,14938.265505
5,Sandwich-NeuralNet,22712,0.007278,NaN,0.000081,12342.529118
6,Sandwich-RandomForest,911440,0.000000,NaN,0.003499,285.819258
7,Sandwich-LogisticRegr,18544,0.014556,NaN,0.000066,15066.676678
